### Importation des identifiants des villes

In [1]:
import pandas as pd

municipalities = pd.read_csv(
    'data/2-0-municipalities.csv',
    index_col=0
)

municipalities

,name
insee_code,
80001,Abbeville
47001,Agen
13001,Aix-en-Provence
2A004,Ajaccio
73011,Albertville
...,...
47323,Villeneuve-sur-Lot
14762,Vire Normandie
51649,Vitry-le-François


### Recherche de données météorologiques

La plupart des applications météorologiques avec API trouvées par une rechercher par mot-clé sont malheureusement payantes. Nous décidons d'avoir alors recours à Wikipedia, au risque de données incomplètes.

La plupart des article de ville sur Wikipédia possèdent en effet une section "climat" qui possède généralement un tableau de variables climatiques moyennes mensuelles / annuelles, notamment la température, les précipitations et . Ce tableau est souvent structuré de la même façon, mais pas systématiquement.

### Extraction proprement dite

La fonction suivante récupère les informations dans les cas les plus classiques.




In [28]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote
import pandas as pd

def read_wiki(city_name, verbose=0):

    # default result with everything NA
    result = {
        "solar_exposure" : pd.NA,
        "precipitation" :  pd.NA,
        "temperature" :    pd.NA,
    }
    
    url = "https://fr.wikipedia.org/wiki/"+quote(city_name)

    with urlopen(url) as page:

        if verbose >= 1:
            print('✅ URL valid', end="")

        parsed_page = BeautifulSoup(page)

        # the id climate is hosted in a span inside a h3 title

        local_climate_title = parsed_page.find(id="Climat")

        if local_climate_title is None :
            print(' 😵 no #climate tag found')
            return(result)

        if verbose >= 1:
            print(' ✅ #climate tag found', end="")
        local_climate_title = local_climate_title.parent

        # the table with temperatures comes after this title,
        # at the same level as the climate title in the page body
        # but BEFORE the next h3 tag

        local_climate_table = None

        for tag in local_climate_title.next_siblings:

            if tag.name=='table' :
                local_climate_table = tag
                if verbose >= 1:
                    print(' ✅ table found', end='')
                break

            if tag.name=='h3' :
                break
        
        # if we found an other h3, it means the section has ended
        # without finding any tables, and we skip the rest

        if tag.name=='h3' :
            print(' 😵 no table found')
            return(result)
        
        local_climate_data = pd.read_html(
            str(local_climate_table),
            decimal=',', thousands=' ',
            index_col = 0
        )[0]
        # pd.read_html() returns a list of tables,
        # here containing only one table
        # (this explains the [0] at the end)
        
        if 'année' not in local_climate_data.columns:

            print(' 😵 no column "année"')
            return(result)
            
        else:

            if 'Précipitations (mm)' in local_climate_data.index:
                print(' ✅ prec', end="")
                prec = local_climate_data.loc['Précipitations (mm)','année']
                if isinstance(prec, str) :
                    result['precipitation'] = float( prec.replace(",", ".").replace('\xa0', "") )
            else :
                print(' ❌ prec', end="")
            
            if 'Ensoleillement (h)' in local_climate_data.index :
                print(' ✅ enso', end="")
                enso = local_climate_data.loc['Ensoleillement (h)','année']
                if isinstance(enso, str) :
                    result['solar_exposure'] = float( enso.replace(",", ".").replace('\xa0', "") )
            else :
                print(' ❌ enso', end="")
            
            if 'Température moyenne (°C)' in local_climate_data.index:
                print(' ✅ temp', end="")
                temp = local_climate_data.loc['Température moyenne (°C)','année']
                if isinstance(temp, str) :
                    result['temperature'] = float( temp.replace(",", ".").replace('\xa0', "") )
            else :
                print(' ❌ temp', end="")
    
    print('')
    return(result)

... et l'on peut tester cette fonction sur la ville d'Angoulême (la page de Poitiers est un peu particulière, en ce sens que le tableau des données climatiques standard est le deuxième de la section, et non pas le premier):

In [29]:
read_wiki("Angoulême")

 ✅ prec ✅ enso ✅ temp


{'solar_exposure': 2089.3, 'precipitation': 843.2, 'temperature': 12.8}

Les problèmes suivants ont été identifiés, dont certains (points 1 et 2) pourraient être traités dans le futur, mais la liste n'est probablement pas exhaustive:

1. le tableau est présent mais pas à la place habituelle (ex: Agen, Poitiers)
2. le tableau est structuré différemment pour permettre à l'utilisateur de le replier (ex: Abbeville)
3. pas de tableau (ex: Autun)
4. pas de colonne année dans le tableau (ex: Lunnéville)

### Systématisation

Dernier obstacle: les pages de Wikipedia peuvent être ambigues. Il existe par exemple plusieurs pages "Argenteuil" et le nom final de la page Wikipédia n'est pas prévisible. Par exemple on aura "Cognac (ville)" (par opposition à "Cognac (boisson)") mais "Argenteuil (Val-d'Oise)". Identifier les bonnes pages est processus essentiellement artisanal.

Le code suivant récupère les donnés pour l'ensemble des communes. (Dans les commentaires, le code INSEE serait nécessaire pour lever l'ambiguité entre Saint-Pierre (Martinique) et Saint-Pierre (La Réunion) ou entre Saint-Denis (La Réunion) et Saint-Denis (Seine-Saint-Denis), mais ces préfectures n'ont pas été traitées par _Marianne_.)

In [20]:
def get_wiki_name(munipality_name) :

    # Python 10 and higher

    # match munipality_name: #
    #    case 'Chaumont':
    #        return("Chaumont (Haute-Marne)")

    elif municipality_name=="Argenteuil" :
        return "Argenteuil (Val-d'Oise)" 
    elif municipality_name=="Aubusson" :
        return "Aubusson (Creuse)" 
    elif municipality_name=="Bergerac" :
        return "Bergerac (Dordogne)" 
    elif municipality_name=="Bernay" :
        return "Bernay (Eure)"             
    elif municipality_name=="Bonnemunicipality_name" :
        return "Bonnemunicipality_name (Haute-Savoie)" 
    if munipality_name=='Chaumont' :
        return "Chaumont (Haute-Marne)"
    elif municipality_name=="Clamecy" :
        return "Clamecy (Nièvre)" 
    elif municipality_name=="Clermont" :
        return "Clermont (Oise)" 
    elif municipality_name=="Cognac" :
        return "Cognac (Charente)" 
    elif municipality_name=="Condom" :
        return "Condom (Gers)" 
    elif municipality_name=="Die" :
        return "Die (Drôme)" 
    elif municipality_name=="Dieppe" :
        return "Dieppe (Seine-Maritime)" 
    elif municipality_name=="Dole" :
        return "Dole (Jura)" 
    elif municipality_name=="Gourdon" :
        return "Gourdon (Lot)" 
    elif municipality_name=="La Trinité" :
        return "La Trinité (Martinique)" 
    elif municipality_name=="Langon" :
        return "Langon (Gironde)" 
    elif municipality_name=="Laval" :
        return "Laval (Mayenne)"          
    elif municipality_name=="Le Blanc" :
        return "Le Blanc (Indre)" 
    elif municipality_name=="Le Vigan" :
        return "Le Vigan (Gard)" 
    elif municipality_name=="Lens" :
        return "Lens (Pas-de-Calais)" 
    elif municipality_name=="Mauriac" :
        return "Mauriac (Cantal)" 
    elif municipality_name=="Mayenne" :
        return "Mayenne (commune)"
    elif municipality_name=="Mende" :
        return "Mende (Lozère)"
    elif municipality_name=="Montdidier" :
        return "Montdidier (Somme)" 
    elif municipality_name=="Montreuil" :
        return "Montreuil (Pas-de-Calais)"
    elif municipality_name=="Moulins" :
        return "Moulins (Allier)"
    elif municipality_name=="Muret" :
        return "Muret (Haute-Garonne)" 
    elif municipality_name=="Neufchâteau" :
        return "Neufchâteau (Vosges)" 
    elif municipality_name=="Prades" :
        return "Prades (Pyrénées-Orientales)" 
    elif municipality_name=="Péronne" :
        return "Péronne (Somme)" 
    elif municipality_name=="Pontoise" :
        return "Pontoise (Chef-lieu)"
    elif municipality_name=="Rochechouart" :
        return "Rochechouart (Haute-Vienne)" 
    elif municipality_name=="Rochefort" :
        return "Rochefort (Charente-Maritime)" 
    elif municipality_name=="Saint-Benoît" :
        return "Saint-Benoît (La Réunion)" 
    elif municipality_name=="Saint-Claude" :
        return "Saint-Claude (Jura)" 
    elif municipality_name=="Saint-Denis" :
        return "Saint-Denis (Seine-Saint-Denis)"
    # elif municipality_name=="Saint-Denis" and municipality_code == :
    #     return "Saint-Denis (La Réunion)"
    elif municipality_name=="Saint-Flour" :
        return "Saint-Flour (Cantal)" 
    elif municipality_name=="Saint-Girons" :
        return "Saint-Girons (Ariège)" 
    elif municipality_name=="Saint-Omer" :
        return "Saint-Omer (Pas-de-Calais)" 
    elif municipality_name=="Saint-Paul" :
        return "Saint-Paul (La Réunion)" 
    # elif municipality_name=="Saint-Pierre" and municipality_code == :
    #     return "Saint-Pierre (Martinique)"
    # elif municipality_name=="Saint-Pierre" and municipality_code == :
    #     return "Saint-Pierre (La Réunion)"
    elif municipality_name=="Senlis" :
        return "Senlis (Oise)" 
    elif municipality_name=="Sens" :
        return "Sens (Yonne)" 
    elif municipality_name=="Torcy" :
        return "Torcy (Seine-et-Marne)" 
    elif municipality_name=="Ussel" :
        return "Ussel (Corrèze)" 
    elif municipality_name=="Valence" :
        return "Valence (Drôme)"
    elif municipality_name=="Vienne" :
        return "Vienne (Isère)"

In [21]:
get_wiki_name("Chaumont")

'Chaumont (Haute-Marne)'

Il est alors possible de requêter les pages Wikipédia les unes après les autres:

In [40]:
weather = []

for code, values in municipalities.iterrows():    

    municipality_name =  values[0]
    wiki_name = get_wiki_name(municipality_name)

    print('📍 '+ municipality_name.ljust(32), end='')
    
    weather.append( read_wiki( wiki_name ) )

📍 Abbeville                        - 😵 no table found
📍 Agen                             - 😵 no column "année"
📍 Aix-en-Provence                  ✅ prec ✅ enso ✅ temp
📍 Ajaccio                          ✅ prec ✅ enso ✅ temp
📍 Albertville                      - 😵 no table found
📍 Albi                             ✅ prec ✅ enso ✅ temp
📍 Alençon                          - 😵 no column "année"
📍 Alès                             ✅ prec ❌ enso ✅ temp
📍 Altkirch                         - 😵 no table found
📍 Ambert                           ✅ prec ✅ enso ✅ temp
📍 Amiens                           ✅ prec ❌ enso ✅ temp
📍 Ancenis-Saint-Géréon             - 😵 no column "année"
📍 Angers                           ✅ prec ✅ enso ✅ temp
📍 Angoulême                        ✅ prec ✅ enso ✅ temp
📍 Annecy                           ✅ prec ✅ enso ✅ temp
📍 Apt                              ✅ prec ❌ enso ❌ temp
📍 Arcachon                         - 😵 no column "année"
📍 Argelès-Gazost                   - 😵 no column "

In [48]:
weather_df = pd.DataFrame(weather)
weather_df.set_index(municipalities.index, inplace=True)
weather_df.to_csv('data/2-3-weather.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f812232-8ed1-4e89-8f48-597e10058637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>